In [2]:
#import necessary modules
#Python >= 3.5
import sys
assert sys.version_info >= (3, 5)

#Scikit-Learn >= 0.20
import sklearn as sk
assert sk.__version__ >= "0.20"

#???
from sklearn.model_selection import StratifiedShuffleSplit

#numpy for calculations
import numpy as np
import os

#matplotlib for figures
import matplotlib as mpl
import matplotlib.pyplot as plt
#a magic function that allows inline plotting so figures are rendered in this notebook
%matplotlib inline

#pandas for ...
import pandas as pd
from pandas.plotting import scatter_matrix

#add other needed modules here
#Tensorflow?

## Setup and Feature Statistics

In [7]:
#define where to save figures for this project
"""
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
"""

'\nPROJECT_ROOT_DIR = "."\nCHAPTER_ID = "end_to_end_project"\nIMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)\nos.makedirs(IMAGES_PATH, exist_ok=True)\n\ndef save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):\n    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)\n    print("Saving figure", fig_id)\n    if tight_layout:\n        plt.tight_layout()\n    plt.savefig(path, format=fig_extension, dpi=resolution)\n'

In [3]:
#define a function to load data from the cv file using pandas
#convert to tensors??? how many dimensions? how many tensors/matrices/vectors?

#call function and store dataframe
scores = pd.read_csv("MCAT_Data.csv", header=1)
#dataTest = pd.read_csv("csv_tests.csv", header=1,usecols=range(6,18))

#sanity check: take a look at the data
#preview first five lines
scores.head()
#print number of columns, column labels, column data types, memory usage, range index, and non-null number of cells in each column
scores.info()
#gives count, mean, standard deviation, min, max, and percentiles (including median)
scores.describe(percentiles = [0.25, 0.5, 0.75], include = 'all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           4676 non-null   object 
 1   Exam Date           4301 non-null   object 
 2   Real Score          4948 non-null   float64
 3   C/P Score           4947 non-null   float64
 4   CARS Score          4947 non-null   float64
 5   B/B Score           4947 non-null   float64
 6   P/S Score           4947 non-null   float64
 7   Days Before Exam    1733 non-null   float64
 8   Total Score         1750 non-null   float64
 9   C/P Score.1         1502 non-null   float64
 10  CARS Score.1        1498 non-null   float64
 11  B/B Score.1         1496 non-null   float64
 12  P/S Score.1         1496 non-null   float64
 13  Days Before Exam.1  2567 non-null   float64
 14  Total Score.1       2633 non-null   float64
 15  C/P Score.2         2330 non-null   float64
 16  CARS S

,Timestamp,Exam Date,Real Score,C/P Score,CARS Score,B/B Score,P/S Score,Days Before Exam,Total Score,C/P Score.1,...,C/P Score.3,CARS Score.3,B/B Score.3,P/S Score.3,Exam Before Exam,Total Score.3,C/P Score.4,CARS Score.4,B/B Score.4,P/S Score.4
count,4676,4301,4948.000000,4947.000000,4947.000000,4947.000000,4947.000000,1733.000000,1750.000000,1502.000000,...,3044.000000,3043.000000,3042.000000,3042.000000,3340,4000,3544,3543,3541,3544
unique,4650,342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,242,49,17,17,18,18
top,9/20/16,1/28/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,28,515,130,128,130,129
freq,14,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,269,272,676,799,796,812
mean,NaN,NaN,515.914511,128.947847,128.163331,129.123711,129.598949,8.417773,515.797714,129.312250,...,128.729632,128.400920,128.880342,127.856673,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,23.805011,6.247823,6.283536,6.259038,6.196021,16.390119,6.227275,1.996091,...,1.988670,2.358199,1.905568,1.745526,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,417.000000,30.000000,29.000000,30.000000,29.000000,0.000000,490.000000,120.000000,...,118.000000,118.000000,118.000000,118.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,512.000000,128.000000,127.000000,128.000000,128.000000,4.000000,512.000000,128.000000,...,127.000000,127.000000,128.000000,127.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,517.000000,129.000000,128.000000,129.000000,130.000000,6.000000,517.000000,130.000000,...,129.000000,129.000000,129.000000,128.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,520.000000,130.000000,130.000000,131.000000,131.000000,7.000000,520.000000,131.000000,...,130.000000,130.000000,130.000000,129.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#plot some data statistics for the report

## Creat a Test Set

In [ ]:
#set a seed so the output is the same every run
np.random.seed(...)

In [31]:
#split data into training and testing and save in variables
#what is the best way to do that for our data set?
#justification needed for Report 1
#look at sklearn function train_test_split()
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
#test size = ???
#random_state = ???
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#Insert data it in an array 
blank_scores = pd.read_csv("MCAT_Data.csv", header=1,usecols=range(3,3+4*6))


blank_scores_array = blank_scores.values
'''
#Reshape them as a tensor of dimension : number of data points, number of blank exams, number of "under exams" + total score + days before exam 
blank_scores_tensor = blank_scores_array.reshape(blank_scores_array.shape[0],4,6)
blank_scores_tensor = blank_scores_tensor[:,:,:3]

X = np.squeeze(blank_scores_tensor,axis=2)
print(X.shape)
y = scores["Real Score"].to_numpy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf = LogisticRegression(solver='liblinear' )
clf.fit(X_train[:200], y_train[:200])

#sanity check: look at how many data points are in train/test
len(train_set)
len(test_set)

'''

for i in range(4):
    blank_scores = blank_scores.drop(columns=["Days Before Exam"+str(i)])
#blank_scores = blank_scores.drop(columns=["Days Before Exam.1"])
#    i+=1
blank_scores.head(5)

KeyError: "['Days Before Exam0'] not found in axis"

In [ ]:
#perform checksum/hash sum OR cyclic rendunancy check (CRC) on data
#how to do it???
#see here: https://www.codeproject.com/Articles/1671/CRC32-Generating-a-checksum-for-a-file

In [2]:
#sanity check: preview test and training data
train_set.head()
test_set.head()

NameError: name 'train_set' is not defined

In [ ]:
#perform stratified shufflesplit cross-validator
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
#compares random and stratified error

## Visualize the Data

In [ ]:
#create scatter plot of data with alpha=???
#add legends, titles, and color

## Visualize Correlations

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html

#create scatter matrices displaying totals, medians..

## Prepare Data for ML Algorithms

In [ ]:
#drop data labels

#save labels in a separate variable

In [ ]:
# drop rows that have negative values
#                # days before exam > 180
#                # no final MCAT score
#                # exact duplicate values to another row

# sanity check: preview the data

In [ ]:
# add custom attribute: variance between test scores (if multiple test scores)
# NOTE: make sure variance is calculated between different tests, not between subsections of the same test

## Transformation Pipeline

In [ ]:
# allows you to easily apply a list of transforms and fit methods in a given order
from sklearn.pipeline import Pipeline
# standardizes features, which is a requirement for many ML estimators
from sklearn.preprocessing import StandardScaler

# Select and Train a Model

## Training and Evaluating on the Training Set

In [ ]:
# NOTE: Explain the process of model validation
# how did you split the data into training, validation and test sets
# What are the sizes of each set and why did you make such design choice.

from sklearn.linear_model import LinearRegression

# initialize a linear regression object
lin_reg = LinearRegression()
lin_reg.fit("DATA", "LABELS")

# sanity check: test out predictions and compare to labels
print(lin_reg.predict("SOME DATA"))
print(list("SOME LABELS"))

In [ ]:
# loss function
from sklearn.metrics import mean_squared_error

MCAT_predictions = lin_reg.predict("DATA")
lin_rmse = mean_squared_error("LABELS", MCAT_predictions, squared=False)
lin_rmse

## Evaluation with Cross-Validation

In [ ]:
lin_scores = cross_val_score(lin_reg, "DATA", "LABELS",
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)